In [263]:
from pymongo import MongoClient
import pprint

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from timeit import default_timer as timer

from bs4 import BeautifulSoup

import json


In [190]:
#Connect to hosted MongoDB instance
client = MongoClient()
#Access already existing database
db = client['capstone2_updated']
#Access already existing collection in database
diamond_info = db['diamond_info']

## Parsing one page first to get workflow before iterating through all pages

In [147]:
diamonds = diamond_info.find_one({'pull':1})['html']
soup = BeautifulSoup(diamonds,'html.parser')

example = []
#getting list of shapes
# for i in range(len(soup.find_all('div','shape-cell-wrapper'))):
for i in range(12,17):
    example_diamonds = dict()
    #getting diamond shape
    example_diamonds['shape'] = soup.find_all('div','shape-cell-wrapper')[i].text
    #getting diamond price - some were discounted
    if len(soup.find_all('div','row-cell price')[i].text.split('Now: ')) == 2:
        og_price = soup.find_all('div','row-cell price')[i].text.split('Now: ')[0][6:].replace(',','')
        example_diamonds['original_price'] = int(og_price)
        disc_price = soup.find_all('div','row-cell price')[i].text.split('Now: ')[1][1:].replace(',','')
        example_diamonds['discounted_price'] = int(disc_price)
    else: 
        og_price = soup.find_all('div','row-cell price')[i].text[1:].replace(',','')
        example_diamonds['original_price'] = int(og_price)
        example_diamonds['discounted_price'] = np.nan
    #getting diamond carat
    example_diamonds['carat'] = soup.find_all('div','row-cell carat')[i].text
    #getting diamond cut
    #repeats the cut 2x with no space in between - need to cut string in half
    x = int(len(soup.find_all('div','row-cell cut')[i].text)/2)
    example_diamonds['cut'] = soup.find_all('div','row-cell cut')[i].text[x:]
    #getting diamond color
    example_diamonds['color'] = soup.find_all('div','row-cell color')[i].text
    #getting diamond clarity
    example_diamonds['clarity'] = soup.find_all('div','row-cell clarity')[i].text
    #getting diamond polish
    example_diamonds['polish'] = soup.find_all('div','row-cell polish')[i].text
    #getting diamond symmetry
    example_diamonds['symmetry'] = soup.find_all('div','row-cell symmetry')[i].text
    #getting diamond fluorescence
    example_diamonds['fluorescence'] = soup.find_all('div','row-cell fluorescence')[i].text
    #getting diamond depth
    example_diamonds['depth'] = soup.find_all('div','row-cell depth')[i].text
    #getting diamond table
    example_diamonds['table'] = soup.find_all('div','row-cell table')[i].text
    #getting diamond lw ratio
    example_diamonds['lxwratio'] = soup.find_all('div','row-cell lxwratio')[i].text
    #getting diamond culet
    example_diamonds['culet'] = soup.find_all('div','row-cell culet')[i].text
    example.append(example_diamonds)
example

[{'shape': 'Oval',
  'original_price': 238,
  'discounted_price': nan,
  'carat': '0.30',
  'cut': 'Very Good',
  'color': 'I',
  'clarity': 'SI1',
  'polish': 'Excellent',
  'symmetry': 'Very Good',
  'fluorescence': 'Faint',
  'depth': '64.6',
  'table': '60.0',
  'lxwratio': '1.32',
  'culet': 'None'},
 {'shape': 'Emerald',
  'original_price': 261,
  'discounted_price': 238,
  'carat': '0.31',
  'cut': 'Good',
  'color': 'K',
  'clarity': 'VVS2',
  'polish': 'Excellent',
  'symmetry': 'Very Good',
  'fluorescence': 'Faint',
  'depth': '70.7',
  'table': '62.0',
  'lxwratio': '1.28',
  'culet': 'None'},
 {'shape': 'Pear',
  'original_price': 251,
  'discounted_price': 238,
  'carat': '0.30',
  'cut': 'Very Good',
  'color': 'J',
  'clarity': 'SI1',
  'polish': 'Very Good',
  'symmetry': 'Very Good',
  'fluorescence': 'None',
  'depth': '62.8',
  'table': '61.0',
  'lxwratio': '1.52',
  'culet': 'None'},
 {'shape': 'Pear',
  'original_price': 239,
  'discounted_price': nan,
  'carat':

## Iterate through all pages and parse to dataframe

In [265]:
def parse_to_df(num_pulls):
    diamond_list = []
    for i in range(1,num_pulls+1):
        diamonds = diamond_info.find_one({'pull':i})['html']
        soup = BeautifulSoup(diamonds,'html.parser')


        for i,d in enumerate(soup.find_all('div','grid-row row')):
            per_diamond = dict()

            per_diamond['shape'] = d.find('div','shape-cell-wrapper').text  


            if len(d.find('div','row-cell price').text.split('Now: ')) == 2:
                og_price = d.find('div','row-cell price').text.split('Now: ')[0][6:].replace(',','')
                per_diamond['original_price'] = int(og_price)
                disc_price = d.find('div','row-cell price').text.split('Now: ')[1][1:].replace(',','')
                per_diamond['discounted_price'] = int(disc_price)
            else: 
                og_price = d.find('div','row-cell price').text[1:].replace(',','')
                per_diamond['original_price'] = int(og_price)
                per_diamond['discounted_price'] = np.nan


            per_diamond['carat'] = d.find('div','row-cell carat').text
            x = int(len(d.find('div','row-cell cut').text)/2)
            per_diamond['cut'] = d.find('div','row-cell cut').text[x:]
            per_diamond['color'] = d.find('div','row-cell color').text
            per_diamond['clarity'] = d.find('div','row-cell clarity').text
            per_diamond['polish'] = d.find('div','row-cell polish').text
            per_diamond['symmetry'] = d.find('div','row-cell symmetry').text
            per_diamond['fluorescence'] = d.find('div','row-cell fluorescence').text
            per_diamond['depth'] = d.find('div','row-cell depth').text
            per_diamond['table'] = d.find('div','row-cell table').text
            per_diamond['lxwratio'] = d.find('div','row-cell lxwratio').text
            per_diamond['culet'] = d.find('div','row-cell culet').text
            diamond_list.append(per_diamond)


    df = pd.DataFrame(diamond_list)
    return df

In [261]:
num_pulls = diamond_info.count()
num_pulls

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


58

In [266]:
start = timer()
diamond_data = parse_to_df(num_pulls)
diamond_data.to_csv(r'/Users/winglau/Desktop/Docker/Capstone2/updated_diamonds.csv')
end = timer()
print(end - start)

371.95557399500103


In [267]:
diamond_data

,shape,original_price,discounted_price,carat,cut,color,clarity,polish,symmetry,fluorescence,depth,table,lxwratio,culet
0,Emerald,221,NaN,0.30,Good,J,VS2,Very Good,Good,None,70.0,61.0,1.33,None
1,Emerald,224,NaN,0.30,Very Good,J,SI1,Excellent,Very Good,Faint,65.9,66.0,1.42,None
2,Emerald,224,NaN,0.30,Very Good,J,SI1,Excellent,Very Good,Faint,66.6,66.0,1.28,None
3,Pear,225,NaN,0.30,Very Good,J,SI1,Very Good,Very Good,Faint,61.7,62.0,1.60,None
4,Pear,226,NaN,0.30,Very Good,J,SI2,Very Good,Very Good,None,62.5,59.0,1.57,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52881,Pear,882083,NaN,10.88,Very Good,D,FL,Excellent,Excellent,Faint,60.9,63.0,1.61,None
52882,Princess,930395,NaN,15.06,Very Good,F,VS1,Excellent,Excellent,None,73.7,71.0,1.03,None
52883,Pear,1030445,NaN,9.52,Very Good,D,FL,Excellent,Very Good,None,62.3,56.0,1.63,None
52884,Pear,1244433,NaN,10.51,Very Good,D,FL,Excellent,Excellent,None,62.0,56.0,1.55,None
